# WAS Data Exploration

This notebook creates interactive visualization tools for exploring the repository of Whole Air Samples (WAS) cans from the NASA Student Airborne Research Program (SARP) field campaigns (2009-2023).

::::{grid}
:gutter: 3

:::{grid-item-card} Scatter Plot
For comparing two species
:::

:::{grid-item-card} Map
For viewing the spatial distribution of a species
:::

::::

## Setup

In [ ]:
!pip install openpyxl -qq

In [ ]:
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import geoviews as gv
import holoviews as hv
import panel as pn

In [ ]:
pd.options.plotting.backend = 'hvplot'

In [ ]:
# read data file
df = pd.read_excel('/home/jovyan/shared-public/NASA-SARP/data/WAS_explorer/data/WAS_SARP_2009_2021_noground.xlsx', skipfooter=6)

In [ ]:
# drop columns of all nan
df = df.dropna(axis=1, how='all')
# add a datetime column
df['datetime'] = pd.to_datetime(df.date.astype(str) + ' ' + df.OpenTime.astype(str))

In [ ]:
# create a list of the numeric columns for use in selection widgets
numeric_cols = []
for index, value in df.dtypes.items():
    if value != 'object':
        numeric_cols.append(index)

Creating and processing geodataframe

In [ ]:
geo_df = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']), 
    crs='epsg:4326')

## Scatter Plot

In [ ]:
x = pn.widgets.Select(name='x', options=numeric_cols, value='CH4 (ppmv)')
y = pn.widgets.Select(name='y', options=numeric_cols, value='AltP_meters')
color_by_select = pn.widgets.Select(name='color by', options=numeric_cols, value='CO2_ppm')

color_min = pn.widgets.FloatInput(name='color_min', value=400, step=1e-1, start=0, end=5000)
color_max = pn.widgets.FloatInput(name='color_max', value=500, step=1e-1, start=0, end=5000)


def points(ds, x, y, colorby, color_min, color_max):
    return ds.data.hvplot.points(
        x=x, y=y, color=colorby, cmap='viridis', hover_cols=['datetime', 'index'], width=800, 
        clim=(color_min, color_max),
    )

pn.Column(pn.Row(pn.Column(x, y, color_by_select), pn.Column(color_min, color_max)),
          hv.Dataset(df).apply(points, x=x, y=y, colorby=color_by_select, color_min=color_min, color_max=color_max))

## Map

In [ ]:
color = pn.widgets.Select(name='color', options=numeric_cols)  # df.columns.tolist())
float_input = pn.widgets.FloatInput(name='FloatInput', value=5., step=1e-1, start=0, end=1000)
size = pn.widgets.IntSlider(start=5, end=15, name='size')

poly_plot = gv.Points(geo_df[['geometry']].dropna()).opts(width=600, height=600, colorbar=True,
                                                                     toolbar='above', tools=['hover'],
                                                                    cmap='inferno', size=12)

gv.tile_sources.OSM * poly_plot